In [ ]:
!pip install aniemore
!pip install moviepy

In [ ]:
!pip install pydub

In [ ]:
import subprocess
import os
import sys
import torch
import moviepy.editor as mp
from aniemore.recognizers.voice import VoiceRecognizer
from aniemore.recognizers.text import TextRecognizer
from aniemore.models import HuggingFaceModel
from transformers import AutoModel, AutoFeatureExtractor, AutoModelForAudioClassification
from pydub import AudioSegment

In [ ]:
labels = ['disgust', 'happiness', 'anger', 'fear', 'enthusiasm', 'neutral', 'sadness']
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
checkpoint = "facebook/wav2vec2-base"

In [ ]:
model = HuggingFaceModel.Voice.Wav2Vec2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
myvideo = mp.AudioFileClip('/content/tests_aniemore_recognizers_src_my_voice.ogg').subclip(0, 20)
myvideo.write_audiofile('/content/audio.ogg')

MoviePy - Writing audio in /content/audio.ogg


MoviePy - Done.


In [ ]:
MyVideo = mp.VideoFileClip('/content/video.mp4').subclip(0,5)
MyAudio = MyVideo.audio
MyAudio.write_audiofile('/content/audio.wav', codec = 'pcm_s32le')

MoviePy - Writing audio in /content/audio.wav


MoviePy - Done.


In [ ]:
vr = VoiceRecognizer(model=model, device=device)



In [ ]:
model = HuggingFaceModel.Text.Bert_Tiny
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# tr - acronym for TextRecognizer from two first capital letters
tr = TextRecognizer(model=model, device=device)

In [ ]:
audio_file = AudioSegment.from_file('/content/video.mp3')
start_time = 1000  # начало обрезки в миллисекундах
end_time = 2000  # конец обрезки в миллисекундах
trimmed_audio = audio_file[start_time:end_time]
trimmed_audio.export("/content/output.ogg", format="ogg")

<_io.BufferedRandom name='/content/output.ogg'>

In [ ]:
result = vr.recognize('/content/output.ogg')

RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [1, 1, 2, 16022]

In [ ]:
text = 'если ты не прекратишь, я тебя убью!'
result = tr.recognize(text, return_single_label=True)

In [ ]:
result

{'anger': 2.5057996026589535e-05,
 'disgust': 2.9086111680953763e-05,
 'enthusiasm': 7.913003355497494e-05,
 'fear': 2.57901683653472e-06,
 'happiness': 1.127377436205279e-05,
 'neutral': 0.9997871518135071,
 'sadness': 6.56655101920478e-05}

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
test_datase = load_dataset("Aniemore/resd_annotated", split="test")



Generating train split:   0%|          | 0/1116 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/280 [00:00<?, ? examples/s]

In [ ]:
test_datase[0]

{'name': '27_neutral_fear_n_100',
 'path': 'neutral_fear_27/27_neutral_fear_n_100.wav',
 'speech': {'path': '27_neutral_fear_n_100.wav',
  'array': array([ 0.00000000e+00, -6.10351562e-05, -1.22070312e-04, ...,
          3.05175781e-05,  3.05175781e-05,  0.00000000e+00]),
  'sampling_rate': 44100},
 'text': 'Вам дадут целый минимальный оклад, но при этом вам не надо будет ходить на работу. Это же счастье. Я бы просто мечтал вот прям сегодня уйти на пенсию. Я так вам завидую.',
 'emotion': 'neutral'}

In [ ]:
inputs = feature_extractor(test_datase[0]['speech']['array'], return_tensors="pt", sampling_rate=16000)

In [ ]:
with torch.no_grad():
  logits = model(**inputs)

In [ ]:
logits

Wav2Vec2BaseModelOutput(last_hidden_state=tensor([[[ 0.1989, -0.0606, -0.6700,  ...,  0.2639,  0.4582, -0.3821],
         [ 0.3196, -0.0197, -0.6578,  ...,  0.2236,  0.5138, -0.3923],
         [ 0.3416, -0.0153, -0.6280,  ...,  0.2098,  0.4459, -0.4062],
         ...,
         [ 0.2558, -0.0245, -0.5811,  ...,  0.2519,  0.3378, -0.4418],
         [ 0.2638,  0.0064, -0.5288,  ...,  0.2167,  0.3946, -0.3769],
         [ 0.1304,  0.1992, -0.4694,  ...,  0.1582,  0.5517,  0.1424]]]), extract_features=tensor([[[ 2.2246e-01,  4.9831e-01,  4.8796e-02,  ..., -4.3633e-01,
          -8.6346e-04, -5.2068e-01],
         [-1.0746e-01,  2.3846e-01, -1.4436e-01,  ...,  1.3355e-02,
           1.2663e-02, -2.1921e-01],
         [-5.5736e-02,  2.1355e-03,  4.1983e-02,  ..., -3.2665e-02,
           1.0711e-02, -2.6137e-01],
         ...,
         [ 5.5366e-02,  4.7059e-01, -1.1778e-01,  ..., -2.8981e-01,
           9.9160e-03,  4.5091e-01],
         [-9.8818e-02,  1.0269e+00,  1.2163e-01,  ...,  4.7047e-